In [1]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [16]:
GRF = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF gtpase code\RF_molecules_Newfeatures_GTPase_KRas.csv")
CRF = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF G12C Code\RF_molecules_Newfeatures_G12C.csv")
DRF = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF G12D code\RF_molecules_Newfeatures_G12D.csv")
GNN = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\\NN GTPase Code\\NN_molecules_Newfeatures_GTPase_KRAS.csv")
CNN = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\\NN G12C Code\\NN_molecules_Newfeatures_G12C.csv")
DNN = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\\NN G12D Code\\NN_molecules_Newfeatures_G12D.csv")

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:3: SyntaxWarning: invalid escape sequence '\T'
<>:4: SyntaxWarning: invalid escape sequence '\T'
<>:5: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:3: SyntaxWarning: invalid escape sequence '\T'
<>:4: SyntaxWarning: invalid escape sequence '\T'
<>:5: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
C:\Users\TheSh\AppData\Local\Temp\ipykernel_2788\3473304193.py:1: SyntaxWarning: invalid escape sequence '\T'
  GRF = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF gtpase code\RF_molecules_Newfeatures_GTPase_KRas.csv")
C:\Users\TheSh\AppData\Local\Temp\ipykernel_2788\3473304193.py:2: SyntaxWarning: invalid escape sequence '\T'
  C

In [18]:
df = DRF
df.head()

,ChEMBL ID,Frequency,Avg Predicted Value
0,CHEMBL1200866,0.72,7.944964
1,CHEMBL1187833,0.54,7.924939
2,CHEMBL46286,0.52,7.924906
3,CHEMBL3544914,0.50,7.932936
4,CHEMBL2448612,0.46,7.922380


In [19]:
list = []
for i in df["ChEMBL ID"]:
    molecule = new_client.molecule
    m1 = molecule.filter(chembl_id=i).only(
        ['molecule_chembl_id', 'pref_name', 'molecule_structures'])
    targets = pd.DataFrame.from_dict(m1)
    #print(targets['molecule_chembl_id'][0])
    if targets['molecule_chembl_id'][0] == i:
        list.append([i,targets['pref_name'][0]])
        print(i,"&", targets['pref_name'][0],"\\\\\n \\hline")

CHEMBL1200866 & PERFLUBRON \\
 \hline
CHEMBL1187833 & UMECLIDINIUM \\
 \hline
CHEMBL46286 & OMACETAXINE MEPESUCCINATE \\
 \hline
CHEMBL3544914 & TEZACAFTOR \\
 \hline
CHEMBL2448612 & ERGOTAMINE TARTRATE \\
 \hline
CHEMBL2364638 & UBROGEPANT \\
 \hline
CHEMBL3989826 & OXYCODONE TEREPHTHALATE \\
 \hline
CHEMBL575 & METHICILLIN \\
 \hline
CHEMBL5314588 & PERFLUOROHEXYLOCTANE \\
 \hline
CHEMBL523299 & UMECLIDINIUM BROMIDE \\
 \hline
